# BPE Tokenizer

Pada kode ini, ditunjukkan contoh penggunaan BPE Tokenizer buatan HuggingFace. [Petunjuk instalasi package](https://huggingface.co/docs/tokenizers/installation)

In [56]:
# Import semua packages yang diperlukan.
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [57]:
# Instantiate tokenizer BPE dengan unknown token [UNK] (berguna 
# saat testing jika menemukan misalnya karakter yang tidak dikenali).
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [58]:
# Instantiate trainer BPE, PERHATIKAN hyperparameter vocab_size yang konsepnya
# mirip dengan `num_of_merges` yang diajarkan di kelas (relasinya kurang lebih 
# num_of_merges = vocab_size - num_of_alphabet).
trainer = BpeTrainer(
    vocab_size=100,  # Perbesar/perkecil dan lihat bedanya pada hasil tokenisasi
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
)

In [59]:
# Menambahkan pre-tokenizer yang berfungsi untuk memecah tiap kalimat/dokumen
# menjadi individual words berdasarkan keberadaan whitespace.
tokenizer.pre_tokenizer = Whitespace()

In [60]:
# Melatih tokenizer dengan corpus.
tokenizer.train(
    files=["idwiki_index.txt"],
    trainer=trainer
)

In [61]:
# Contoh melakukan tokenisasi suatu kalimat
tokenizer.encode('Azhar Mansor').tokens

['A', 'z', 'h', 'ar', 'M', 'an', 's', 'o', 'r']